# Implementation of the Algorithm Using Torch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
from matching.algorithms import galeshapley

In [ ]:
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')
ctgan = pd.read_csv('Syn data/ctgan_5000.csv')
gauscop = pd.read_csv('Syn data/gauscop_5000.csv')
synthpop = pd.read_csv('Syn data/synthpop_5000.csv')
tvae = pd.read_csv('Syn data/tvae_5000.csv')
syndata = pd.read_csv('synData9.csv')
realdata34 = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

realdata34.drop('STDs: Time since first diagnosis', axis=1, inplace=True)
realdata34.drop('STDs: Time since last diagnosis', axis=1, inplace=True)

In [ ]:
med34 = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis']

med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis',
        'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']

bin = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis',
        'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
        'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
        'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
        'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
        'STDs:Hepatitis B', 'STDs:HPV','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
        'Citology', 'Biopsy']

In [ ]:
def impute(data, graph=False):
    temp_data = data
    temp_data[temp_data == '?'] = np.nan

    for i in temp_data.columns:
        if(temp_data[i].dtype == 'O'):
            temp_data[i] = temp_data[i].astype('float')
    if len(data.columns) == 34:
        med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis']
    else:
        med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis',
        'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']
        
    for i in med:
        imp = float(temp_data[i].median())
        temp_data[i].fillna(imp, inplace=True)
        temp_data[i].replace(np.nan, imp)

    bin = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis',
        'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
        'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
        'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
        'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
        'STDs:Hepatitis B', 'STDs:HPV','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
        'Citology', 'Biopsy']

    for i in bin:
        imp = float(temp_data[i].mode())
        temp_data[i].fillna(imp, inplace=True)
        temp_data[i].replace(np.nan, imp)

    if graph:
        temp_data.hist(figsize=(25,15))
    
    return temp_data

def norm(data1):
    """
    Min-Max normalizes the continuous data
    """
    new_dat = data1.copy()
    for col in new_dat.columns:
        if col in med:
            denom = new_dat[col].max() - new_dat[col].min()
            if denom != 0:
                new_dat[col] = (new_dat[col] - new_dat[col].min()) / denom
            else:
                new_dat[col] = 0
    return new_dat

In [ ]:
realdataA = norm(impute(realdata))
syndataA = norm(impute(syndata))
ctganA = norm(impute(ctgan))
gauscopA = norm(impute(gauscop))
synthpopA = norm(impute(synthpop))
tvaeA = norm(impute(tvae))
realdata34A = norm(impute(realdata34))

syndata.isnull().values.any(), realdata34.isnull().values.any(), realdata.isnull().values.any(), ctganA.isnull().values.any(), gauscopA.isnull().values.any(), synthpopA.isnull().values.any(), tvaeA.isnull().values.any()

In [ ]:
def compute_distance(df1, df2):
    
    ar1 = df1.to_numpy()
    ar2 = df2.to_numpy()
    x1 = torch.Tensor(ar1)
    x2 = torch.Tensor(ar2)

    y1 = x1.unsqueeze(2).expand(x1.size()[0], x1.size()[1], x2.size()[0])
    y2 = x2.unsqueeze(2).expand(x2.size()[0], x2.size()[1], x1.size()[0]).permute(2,1,0)

    weights = torch.ones(x1.size()[1])

    Δy = torch.abs(y1-y2)

    result = torch.matmul(Δy.permute(0,2,1), weights).numpy()
    
    return result

In [ ]:
syndataB = compute_distance(realdataA, syndataA)
ctganB = compute_distance(realdata34A, ctganA)
gauscopB = compute_distance(realdata34A, gauscopA)
synthpopB = compute_distance(realdata34A, synthpopA)
tvaeB = compute_distance(realdata34A, tvaeA)
realvreal = compute_distance(realdataA, realdataA)
real34vreal34 = compute_distance(realdata34A, realdata34A)

In [ ]:
newSyn = []
newCt = []
newGaus = []
newSynth = []
newTvae = []
realT = []

for i in range(858):
    newSyn.append(syndataB[i][:858].tolist())
    newCt.append(ctganB[i][:858].tolist())
    newGaus.append(gauscopB[i][:858].tolist())
    newSynth.append(synthpopB[i][:858].tolist())
    newTvae.append(tvaeB[i][:858].tolist())
    realT.append(realvreal[i][:858].tolist())

## Graphs

In [ ]:
plt.imshow(real34vreal34[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(realvreal[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(syndataB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(ctganB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(gauscopB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(synthpopB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(tvaeB[:858,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

## Ranking


In [ ]:
def createDict(A):
    s = torch.argsort(torch.Tensor(A),dim=1)
    d = {}
    for i in range(len(A)):
        d[i] = s[i,:].tolist()
    return d

In [ ]:
newSynM = newSyn

realDict = createDict(newSynM)
synDict = createDict(torch.tensor(newSynM).T)

matchNewSyn = galeshapley(realDict, synDict)

In [ ]:
y1 = []
for i in range(len(newSyn)):
    y1.append(newSyn[i][matchNewSyn[i]])
x1 = np.array([i for i in range(858)])

plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Real Data vs Score for Syndata")

plt.scatter(x1, y1)
plt.show()

In [ ]:
newCtM = newCt 
newGausM = newGaus
realTM = realT
newTvaeM = newTvae
newSynthM = newSynth



realDict2 = createDict(newCtM)
synDict2 = createDict(torch.tensor(newCtM).T)

realDict3 = createDict(newGausM)
synDict3 = createDict(torch.tensor(newGausM).T)

realDictR = createDict(realTM)
synDictR = createDict(torch.tensor(realTM).T)

realDict4 = createDict(newTvaeM)
synDict4 = createDict(torch.tensor(newTvaeM).T)

realDict5 = createDict(newSynthM)
synDict5 = createDict(torch.tensor(newSynthM).T)

In [ ]:
matchingR = galeshapley(realDictR, synDictR)

In [ ]:
y2 = []
for i in range(len(realT)):
    y2.append(realT[i][matchingR[i]])
x2 = np.array([i for i in range(858)])

plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Real Data vs Score for Real Data")

plt.scatter(x2, np.sort(y2))
plt.show()

In [ ]:
matching2 = galeshapley(realDict2, synDict2)
matching3 = galeshapley(realDict3, synDict3)
matching4 = galeshapley(realDict4, synDict4)
matching5 = galeshapley(realDict5, synDict5)

In [ ]:
y3 = []
for i in range(len(newCt)):
    y3.append(newCt[i][matching2[i]])
x3 = np.array([i for i in range(858)])

plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Real Data vs Score for CTGan")

plt.scatter(x3, y3)
plt.show()

In [ ]:
y4 = []
for i in range(len(newGaus)):
    y4.append(newGaus[i][matching3[i]])
x4 = np.array([i for i in range(858)])

plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Real Data vs Score for GAUSCOP")

plt.scatter(x4, y4)
plt.show()

In [ ]:
y5 = []
for i in range(len(newTvae)):
    y5.append(newTvae[i][matching4[i]])
x5 = np.array([i for i in range(858)])

plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Real Data vs Score for Tvae")

plt.scatter(x5, y5)
plt.show()

In [ ]:
y6 = []
for i in range(len(newSynth)):
    y6.append(newSynth[i][matching5[i]])
x6 = np.array([i for i in range(858)])

plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Real Data vs Score for Synthpop")

plt.scatter(x6, y6)
plt.show()

In [ ]:
plt.xlabel('Index of Real Patient') 
plt.ylabel('Score') 
plt.title("Plot of all the Scores for Every Synthetic Method")
plt.scatter(x2, y2, c='lightblue')
plt.scatter(x1, y1, c='red')
plt.scatter(x3, y3, c='green')
plt.scatter(x4, y4, c='blue')
plt.scatter(x5, y5, c='purple')
plt.scatter(x6, y6, c='yellow')
plt.legend(['Real Data', 'Syndata', 'CTGan', 'GAUSCOP', 'Tvae', 'Synthpop'])
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (10,7.5)
plt.ylabel('Score') 
plt.ylabel('Patients')
plt.title("Sorted Plot of all the Scores for Every Synthetic Method")
plt.scatter(x2, np.sort(y2), c='lightblue')
plt.scatter(x1, np.sort(y1), c='red')
plt.scatter(x3, np.sort(y3), c='green')
plt.scatter(x4, np.sort(y4), c='blue')
plt.scatter(x5, np.sort(y5), c='purple')
plt.scatter(x6, np.sort(y6), c='yellow')
plt.legend(['Real Data', 'Syndata', 'CTGan', 'GAUSCOP', 'Tvae', 'Synthpop'])
resolution_value = 1200
plt.savefig("myImage.png", format="png", dpi=resolution_value)
plt.show()

## Choice Graphs

Check preferences. For patient 0, we want to check if the final choice it got was first choice, second choice, third choice, or greater than third choice.

In [ ]:
newCtM = newCt 
newGausM = newGaus
realTM = realT
newTvaeM = newTvae
newSynthM = newSynth
newSynM = newSyn

In [ ]:
dataset_used = newCt

newSynM = dataset_used

realDict = createDict(newSynM)
synDict = createDict(torch.tensor(newSynM).T)
matchNewSynM = galeshapley(realDict, synDict)

In [ ]:
dictUsed = realDict
matchUsed = matchNewSynM

In [ ]:
y = []
for i in range(len(dataset_used)):
    y.append(dataset_used[i][matchNewSynM[i]])
x = np.array([i for i in range(858)])
plt.scatter(x, np.sort(y))
plt.show()

In [ ]:
def choiceMap(rank, match):
    rankNew = rank
    matchNew = match
    d = {}
    for i in range(len(matchNew)):
        x = matchNew[i]
        x = rankNew[i].index(x)
        d[i] = x
    return d

In [ ]:
map = choiceMap(dictUsed, matchUsed)

In [ ]:
plt.scatter(map.keys(), map.values())
plt.show()

In [ ]:
testPlot = map
X = list(testPlot.keys())
Y = list(testPlot.values())

color_lst = []
colors_set = {0:'green', 1:'blue', 2:'yellow', 3:'red', 4:'black'}

for i, j in enumerate(X):
    if Y[i] <= 3:
        color_lst.append(Y[i])
    if Y[i] > 3:
        color_lst.append(4)

    plt.scatter(X[i], Y[i], color = colors_set.get(color_lst[i], 'black'))

For all the plots on one single plot

In [ ]:
newCtM = newCt 
newGausM = newGaus
realTM = realT
newTvaeM = newTvae
newSynthM = newSynth
newSynM = newSyn

In [ ]:
def gale_shap_plot(dataset_for_use, title=None):

    realDict = createDict(dataset_for_use)
    synDict = createDict(torch.tensor(dataset_for_use).T)
    matchdataset_for_use = galeshapley(realDict, synDict)

    dictUsed = realDict
    matchUsed = matchdataset_for_use

    y = []
    for i in range(len(dataset_for_use)):
        y.append(dataset_for_use[i][matchdataset_for_use[i]])
    x = np.array([i for i in range(858)])
    plt.scatter(x, np.sort(y))
    if title:
        plt.title("%s Method Sorted" % (title))
    plt.ylabel('Score')
    plt.show()

    map = choiceMap(dictUsed, matchUsed)
    plt.scatter(map.keys(), map.values())
    plt.xlabel('Index')
    plt.ylabel('Rank')
    if title:
        plt.title("%s Method; Gale Shapley Algorithm Preference Graph" % (title))
    plt.show()

    testPlot = map
    X = list(testPlot.keys())
    Y = list(testPlot.values())

    color_lst = []
    colors_set = {0:'green', 1:'blue', 2:'yellow', 3:'red', 4:'black'}

    for i, j in enumerate(X):
        if Y[i] <= 3:
            color_lst.append(Y[i])
        if Y[i] > 3:
            color_lst.append(4)

        plt.scatter(X[i], Y[i], color = colors_set.get(color_lst[i], 'black'))
    plt.xlabel('Index')
    plt.ylabel('Rank')
    if title:
        plt.title("%s Method; Gale Shapley Algorithm Preference Graph" % (title))
    plt.show()

In [ ]:
gale_shap_plot(newCtM, title="CTGan")

In [ ]:
gale_shap_plot(newGaus, title="Gauscop")


In [ ]:
gale_shap_plot(realT, title="Real")

In [ ]:
gale_shap_plot(newTvae, "TVAE")

In [ ]:
gale_shap_plot(newSynth, "Synthpop")

In [ ]:
gale_shap_plot(newSyn, "Syndata")